In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql
SELECT 
     name,
     type
        FROM sqlite_master
        WHERE type IN ("table", "view");

Done.


[('album', 'table'),
 ('artist', 'table'),
 ('customer', 'table'),
 ('employee', 'table'),
 ('genre', 'table'),
 ('invoice', 'table'),
 ('invoice_line', 'table'),
 ('media_type', 'table'),
 ('playlist', 'table'),
 ('playlist_track', 'table'),
 ('track', 'table')]

In [7]:
%%sql
SELECT *
    FROM playlist
    limit 10;

Done.


[(1, 'Music'),
 (2, 'Movies'),
 (3, 'TV Shows'),
 (4, 'Audiobooks'),
 (5, '90’s Music'),
 (6, 'Audiobooks'),
 (7, 'Movies'),
 (8, 'Music'),
 (9, 'Music Videos'),
 (10, 'TV Shows')]

In [40]:
%%sql 
WITH combined_table AS
(SELECT
       c.country country,
       g.name name,
       il.quantity,
       il.invoice_line_id invoice_line_id
from customer as c
   INNER JOIN invoice i ON i.customer_id = c.customer_id
   INNER JOIN invoice_line il ON il.invoice_id = i.customer_id
   INNER JOIN track t ON t.track_id = il.track_id
   INNER JOIN genre g ON g.genre_id = t.genre_id
)
SELECT ct.name GENRE_NAME,
       ct.country COUNTRY_NAME,
       SUM(ct.invoice_line_id) TRACKS_SOLD,
    CAST(COUNT(ct.invoice_line_id) AS FLOAT)/
     (SELECT COUNT(*) FROM combined_table) AS PERCENTAGE_OF_TRACKS_SOLD
        FROM combined_table AS ct
        WHERE COUNTRY_NAME = "USA"
        GROUP BY 1
        ORDER BY 3 DESC
        ;
       


Done.


[('Rock', 'USA', 100227, 0.13220551378446116),
 ('Alternative & Punk', 'USA', 11927, 0.015455304928989139),
 ('Metal', 'USA', 11711, 0.017752715121136173),
 ('R&B/Soul', 'USA', 8726, 0.011904761904761904),
 ('Latin', 'USA', 3716, 0.005847953216374269),
 ('Reggae', 'USA', 1870, 0.0020885547201336674),
 ('Heavy Metal', 'USA', 1764, 0.0018796992481203006),
 ('Jazz', 'USA', 1764, 0.002506265664160401),
 ('Classical', 'USA', 1746, 0.0018796992481203006),
 ('Alternative', 'USA', 1716, 0.002506265664160401),
 ('Blues', 'USA', 1692, 0.002506265664160401),
 ('Electronica/Dance', 'USA', 1661, 0.002297410192147034)]

### Based on the above findings we should look out for albums with the ROCK, PUNK and METAL genre as they are the highest when it comes to sales in by genre.

In [53]:
%%sql
WITH sales_per_support_rep AS
(SELECT   
         c.support_rep_id,
         c.first_name ||" "||c.last_name customer_name,
         SUM(i.total) total
    FROM customer AS c
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 2)
SELECT sps.customer_name CUSTOMER,
       e.first_name||" "||e.last_name CUSTOMER_SUPPORT_AGENT,
       sps.total TOTAL_CUSTOMER_PURCHASE
        FROM sales_per_support_rep sps
        INNER JOIN employee e ON e.employee_id = sps.support_rep_id
        GROUP BY 2
        ORDER BY 3 DESC;
       
       


Done.


[('Wyatt Girard', 'Jane Peacock', 99.99),
 ('Victor Stevens', 'Steve Johnson', 76.23),
 ('Stanisław Wójcik', 'Margaret Park', 76.22999999999999)]

In [60]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
         ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


[('USA', 13, 1040.490000000008, 7.942671755725252, 80.03769230769292),
 ('Canada', 8, 535.5900000000034, 7.047236842105309, 66.94875000000043),
 ('Brazil', 5, 427.68000000000245, 7.011147540983647, 85.53600000000048),
 ('France', 5, 389.0700000000021, 7.781400000000042, 77.81400000000042),
 ('Germany', 4, 334.6200000000016, 8.161463414634186, 83.6550000000004),
 ('Czech Republic', 2, 273.24000000000103, 9.108000000000034, 136.62000000000052),
 ('United Kingdom', 3, 245.52000000000078, 8.768571428571457, 81.84000000000026),
 ('Portugal', 2, 185.13000000000022, 6.3837931034482835, 92.56500000000011),
 ('India', 2, 183.1500000000002, 8.72142857142858, 91.5750000000001),
 ('Other', 15, 1094.9400000000085, 7.448571428571486, 72.99600000000056)]